In [60]:
using CSV, DataFrames, Gadfly, Distributions, SparseArrays, LinearAlgebra

include("../iGMRF.jl");
include("../dataGen.jl");

# Création de la grille à 9 cellules

Chaque cellule est représentée par un triplet $[\mu, \sigma, \xi]$ indiquant les vraie valeur des paramètres de la GEV à déterminer.

In [61]:
# Seed
Random.seed!(400)
# Dimensions de la grille
m₁ = 3;
m₂ = 3;
# Nb total de cellules
m = m₁ * m₂
# Hyperparamètres
κᵤ = 100.0;
# Matrice de structure
F = iGMRF(m₁, m₂, κᵤ);
# Grille cible
grid_target = generateTargetGrid(F);
grid_target[:, :, 1] = grid_target[:, :, 1] .+ 10;

In [ ]:
grid_target[:, :, 1]

In [ ]:
grid_target[:, :, 2]

In [ ]:
grid_target[:, :, 3]

# Génération de données

Le but est de générer des données sur chaque cellule à partir des paramètres de la GEV de la grille cible.

In [65]:
nobs = 100;
data = generateData(grid_target, nobs);

In [ ]:
data

# Modèle 

In [ ]:
include("markovHyp.jl");

niter = 10000
δ² = 0.2
κᵤ₀ = 10
μ₀ = zeros(m)

θ = gibbs(niter, data, m₁=m₁, m₂=m₂, δ²=δ², κᵤ₀=κᵤ₀, μ₀=μ₀);
κᵤ = θ[:, 1, 1];
μ = θ[:, 2:end, 1];

warming_size = Int(0.2 * niter);

θsampling = θ[warming_size:end, :, :];
κ̂ᵤ = mean(θsampling.value[:, 1, 1]);
μ̂ = mean(θsampling.value[:, 2:end, 1], dims=1);

In [ ]:
changerate(θ)

In [ ]:
# Trace de μ₁

plot(y=θ.value[:, 2, 1], Geom.line)

In [ ]:
# Phase d'échantillonnage de μ₁

plot(y=θsampling.value[:, 2, 1], Geom.line)

In [ ]:
# Densité expérimentale de μ₁

plot(x=θsampling.value[:, 2, 1], Geom.histogram)

In [ ]:
# Trace de κᵤ

plot(y=θ.value[:, 1, 1], Geom.line)

In [ ]:
# Densité expérimentale de κᵤ

plot(x=θsampling.value[:, 1, 1], Geom.histogram)

In [ ]:
κ̂ᵤ

In [ ]:
summarystats(θsampling)

In [ ]:
reshape(μ̂, m₁, m₂)'

In [ ]:
grid_target[:, :, 1]

Distance entre les deux matrices

In [ ]:
norm(reshape(μ̂, m₁, m₂)' .- grid_target[:, :, 1], 2) / m

In [ ]:
describe(θsampling)